In [2]:
import pandas as pd
import re
import numpy as np
import os
from fuzzywuzzy import fuzz
import ast
from nltk import ngrams

In [3]:
df_food_posts = pd.read_csv(r'..\..\data\data\food_posts_preprocessed\food_posts_preprocessed_1.csv')
len(df_food_posts)

563272

In [4]:
df_food_posts.head()

,Unnamed: 0,id,url,datetime,title,author,num_comments,thumbnail,domain,preprocessed_title,preprocessed_title_nouns,preprocessed_title_nouns_no_duplicates,preprocessed_title_nouns_final
0,0,810ib4,https://i.redd.it/ynh08qx9b1j01.jpg,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,Mike11888,125,https://b.thumbs.redditmedia.com/lEroJcEue-KPY...,i.redd.it,ate thinlysliced japanese wagyu,"['ate', 'wagyu']",['wagyu'],['wagyu']
1,1,810ljv,https://i.redd.it/kobpdjsed1j01.jpg,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",Chef6288,0,default,i.redd.it,smoked brisket doughnuts happened prochef,"['brisket', 'doughnuts', 'prochef']","['brisket', 'doughnuts']","['brisket', 'doughnuts']"
2,2,810lvl,https://i.redd.it/ddugaidod1j01.jpg,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,danceyourdeath,0,https://b.thumbs.redditmedia.com/EA7TutwVLakIt...,i.redd.it,ate matcha mochi melonpan,"['ate', 'matcha', 'mochi', 'melonpan']","['matcha', 'mochi', 'melonpan']","['matcha', 'mochi', 'melonpan']"
3,3,810lx7,https://i.redd.it/yqax4gypd1j01.jpg,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,[deleted],0,default,i.redd.it,wakandablack panther inspired unicorn cake,"['wakandablack', 'panther', 'cake']","['wakandablack', 'panther', 'cake']","['wakandablack', 'panther', 'cake']"
4,4,810m58,https://imgur.com/m9BDe1F,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],PassionateSizzle,1,default,imgur.com,homemade flax seed dough caper homemade,"['homemade', 'flax', 'seed', 'caper', 'homemade']","['flax', 'seed', 'caper']","['flax', 'seed', 'caper']"


In [ ]:
df_food_posts.iloc[0]['preprocessed_title']
string = df_food_posts.iloc[0]['preprocessed_title']
splitString = string.replace(' ', '+')
print(splitString)

In [ ]:
new_df = pd.read_csv('nutritional_information_combined.csv')

In [ ]:
new_df.head()

In [5]:
import json
import math

In [6]:
import requests
from bs4 import BeautifulSoup


In [7]:
import requests
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
        'Pragma': 'no-cache'
    }

In [8]:
def fetch_raw(recipe_url):
    html = None
    title = ''
    serving_size, calories_val, protein_val, fat_val, sugar_val, carb_val, sodium_val, choles_val, fiber_val = 0,0,0,0,0,0,0,0,0
    try:
        r = requests.get(recipe_url, headers=headers)
        if r.status_code == 200:
            html = r.text
            soup = BeautifulSoup(html, 'lxml')
            title = str(soup.select('title')[0]).replace('<title>', '').replace(' | Allrecipes</title>', '')
            json_p = soup.select('.nutrition-row')

            data = json.loads(soup.find('script', type='application/ld+json').text)
            nutrition = data[1]['nutrition']
            
            calories_val = nutrition['calories']
            protein_val = nutrition['proteinContent']
            fat_val = nutrition['fatContent']
            sugar_val = nutrition['sugarContent']
            carb_val = nutrition['carbohydrateContent']
            sodium_val = nutrition['sodiumContent']
            choles_val = nutrition['cholesterolContent']
            fiber_val = nutrition['fiberContent']
            serving_size = nutrition['servingSize']
            
    except Exception as ex:
        print('Exception while accessing raw html')
        print(str(ex))
    finally:
        return {'title': title, 'nutritional_values': {'calories': calories_val, 'serving_size': serving_size,
                                                      'protein': protein_val, 'fat': fat_val, 'sugar':sugar_val,
                                                      'carbohydrates': carb_val, 'sodium': sodium_val, 'cholesterol': choles_val,
                                                      'fiber': fiber_val}}

In [9]:
def get_recipes(title, date):
    serving_size, calories_val, protein_val, fat_val, sugar_val, carb_val, sodium_val, choles_val, fiber_val = 0,0,0,0,0,0,0,0,0
    calories_unit, protein_unit, fat_unit, sugar_unit, carb_unit, sodium_unit, choles_unit, fiber_unit = '', '','','','','','',''
       
    df_new_row = pd.DataFrame(columns = ['food_post_title', 'food_post_date', 'closest_match_from_allrecipes', 
                                                     'calories_allrecipes', 'calories_unit', 'protein_allrecipes', 'protein_unit',
                                                         'fat_allrecipes', 'fat_unit', 'sugar_allrecipes', 'sugar_unit', 
                                                         'carbohydrates_allrecipes', 'carbohydrates_unit', 'sodium_allrecipes', 
                                                         'sodium_unit','cholesterol_allrecipes', 'cholesterol_unit', 
                                                   'fiber_allrecipes', 'fiber_unit'])
    #print(title)
    if title is not None and title != '':   
        title1 = title.split(' ')
        if 'ate' in title1:
            title1.remove('ate')
        if 'eat' in title1:
            title1.remove('eat')

        title = ' '.join(title1)
        edited_title = title.replace(' ', '+')
        recipe_titles, recipes = [], []
        index = 1
        url = 'https://www.allrecipes.com/search/results/?search=' + edited_title

        try:
            r = requests.get(url, headers=headers)
            if r.status_code == 200:
                html = r.text
                soup = BeautifulSoup(html, 'lxml')
                links = soup.select('.card__imageContainer a')
                idx = 0
                if len(links) > 20:
                    links = links[:20]
                for link in links:
                    if link['href'] != "javascript:void(0);":
                        recipe = fetch_raw(link['href'])
                        recipe_titles.append(recipe['title'])
                        if recipe['nutritional_values']['serving_size'] is not None and float(recipe['nutritional_values']['serving_size']) != float(100):
                            print('index: ' + str(index))
                            index = 100/float(recipe['nutritional_values'])

                        if recipe['nutritional_values']['calories'] is not None:
                            calories_val += float(recipe['nutritional_values']['calories'].split(' ')[0])*index
                            calories_unit = recipe['nutritional_values']['calories'].split(' ')[1]
                        
                        if recipe['nutritional_values']['protein'] is not None:
                            protein_val += float(recipe['nutritional_values']['protein'].split(' ')[0])*index
                            protein_unit = recipe['nutritional_values']['protein'].split(' ')[1]
                        
                        if recipe['nutritional_values']['fat'] is not None:
                            fat_val +=float(recipe['nutritional_values']['fat'].split(' ')[0])*index
                            fat_unit = recipe['nutritional_values']['fat'].split(' ')[1]
                            
                        if recipe['nutritional_values']['sugar'] is not None:
                            sugar_val +=float(recipe['nutritional_values']['sugar'].split(' ')[0])*index
                            sugar_unit = recipe['nutritional_values']['sugar'].split(' ')[1]
                        
                        if recipe['nutritional_values']['carbohydrates'] is not None:
                            carb_val =+float(recipe['nutritional_values']['carbohydrates'].split(' ')[0])*index
                            carb_unit = recipe['nutritional_values']['carbohydrates'].split(' ')[1]
                        
                        if recipe['nutritional_values']['sodium'] is not None:
                            sodium_val =+float(recipe['nutritional_values']['sodium'].split(' ')[0])*index
                            sodium_unit = recipe['nutritional_values']['sodium'].split(' ')[1]
                        
                        if recipe['nutritional_values']['cholesterol'] is not None:
                            choles_val =+float(recipe['nutritional_values']['cholesterol'].split(' ')[0])*index
                            choles_unit = recipe['nutritional_values']['cholesterol'].split(' ')[1]
                        
                        if recipe['nutritional_values']['fiber'] is not None:
                            fiber_val=+float(recipe['nutritional_values']['fiber'].split(' ')[0])*index
                            fiber_unit = recipe['nutritional_values']['fiber'].split(' ')[1]      

                        idx+=1
                
                if idx != 0:
                    calories_val = round(calories_val/idx, 2)
                    protein_val = round(protein_val/idx,2)
                    fat_val = round(fat_val/idx,2)
                    sugar_val = round(sugar_val/idx,2)
                    carb_val = round(carb_val/idx,2)
                    sodium_val = round(sodium_val/idx,2)
                    choles_val = round(choles_val/idx,2)
                    fiber_val= round(fiber_val/idx,2)

                
        except Exception as ex:
            print('Exception while accessing raw html')
            print(str(ex))
    
    df_new_row = pd.DataFrame({'food_post_title' : [title], 'food_post_date': [date], 
                                           'closest_match_from_allrecipes': [recipe_titles], 
                                                     'calories_allrecipes': [calories_val], 'calories_unit': [calories_unit], 
                                           'protein_allrecipes': [protein_val],
                                           'protein_unit': [protein_unit],
                                                         'fat_allrecipes': [fat_val], 'fat_unit': [fat_unit], 'sugar_allrecipes': [sugar_val], 
                                           'sugar_unit': [sugar_unit], 'carbohydrates_allrecipes': [carb_val], 'carbohydrates_unit': [carb_unit], 
                                           'sodium_allrecipes': [sodium_val], 
                                                         'sodium_unit': [sodium_unit],'cholesterol_allrecipes': [choles_val], 
                                           'cholesterol_unit': [choles_unit], 
                                                         'fiber_allrecipes': [fiber_val], 'fiber_unit': [fiber_unit]})
    return df_new_row

In [12]:
df_food_titles_similar = pd.DataFrame(columns = ['food_post_title', 'food_post_date', 'closest_match_from_allrecipes', 
                                                 'calories_allrecipes', 'calories_unit', 'protein_allrecipes', 'protein_unit',
                                                     'fat_allrecipes', 'fat_unit', 'sugar_allrecipes', 'sugar_unit', 
                                                     'carbohydrates_allrecipes', 'carbohydrates_unit', 'sodium_allrecipes', 
                                                     'sodium_unit','cholesterol_allrecipes', 'cholesterol_unit', 
                                                     'fiber_allrecipes', 'fiber_unit'])

In [38]:
for index, row in df_food_posts[900:1000].iterrows():
    print(index)
    # Creating empty variables to compute nutritional values
    counter, calories_val, protein_val, fat_val, sugar_val, carb_val, sodium_val, choles_val, fiber_val = 0,0,0,0,0,0,0,0,0
    
    # Creating empty arrays to store information for matching strings in the usda files + their indexes (just for information)
    title_matches = []
    
    # Store the current post title to a temporary variable
    df_new_row = get_recipes(row['preprocessed_title'], row['datetime'])
    
    df_food_titles_similar = pd.concat([df_food_titles_similar, df_new_row])  

900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
Exception while accessing raw html
'NoneType' object is not subscriptable
index: 1
Exception while accessing raw html
float() argument must be a string or a real number, not 'dict'
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
Exception while accessing raw html
'NoneType' object is not subscriptable
index: 1
Exception while accessing raw html
float() argument must be a string or a real number, not 'dict'
987
988
989
990
991
992
993
994
995
996
997
998
999


In [39]:
df_food_titles_similar.head()

,food_post_title,food_post_date,closest_match_from_allrecipes,calories_allrecipes,calories_unit,protein_allrecipes,protein_unit,fat_allrecipes,fat_unit,sugar_allrecipes,sugar_unit,carbohydrates_allrecipes,carbohydrates_unit,sodium_allrecipes,sodium_unit,cholesterol_allrecipes,cholesterol_unit,fiber_allrecipes,fiber_unit
0,thinlysliced japanese wagyu,2018-03-01 00:01:28,"[Japanese Curry Recipe, Japanese Cheesecake Re...",304.6,calories,17.75,g,17.84,g,9.54,g,0.74,g,72.59,mg,5.21,mg,0.02,g
0,smoked brisket doughnuts happened prochef,2018-03-01 00:13:38,"[KC's Smoked Brisket Recipe, Smoked Brisket Ru...",315.34,calories,22.8,g,16.42,g,13.55,g,2.2,g,94.63,mg,9.21,mg,0.15,g
0,matcha mochi melonpan,2018-03-01 00:14:56,"[Matcha Madness Recipe, Matcha Pancakes Recipe...",235.73,calories,5.44,g,7.79,g,17.3,g,7.96,g,16.89,mg,7.17,mg,0.22,g
0,wakandablack panther inspired unicorn cake,2018-03-01 00:15:10,"[Chocolate Unicorn Cake Recipe, Pink Panther R...",414.15,calories,7.11,g,18.39,g,33.42,g,5.25,g,133.15,mg,1.07,mg,1.16,g
0,homemade flax seed dough caper homemade,2018-03-01 00:16:01,"[Homemade Flax Seed Milk Recipe, Flax Seed Cra...",219.42,calories,6.39,g,6.51,g,5.16,g,2.27,g,16.86,mg,0.41,mg,0.31,g


In [41]:
df_food_titles_similar.to_csv('allrecipes_test.csv')

In [ ]:
rec = get_recipes('broccoli soup')